# Einleitung:

### Für unser Python-Projekt haben wir uns für den API der Website "OpenLigaDB" entschieden. Die Website ist eine öffentliche Datenbank für Sportdaten von verschiedenen Sportarten und einer Vielzahl von Ligen. Die Website bietet einen API mit mehreren Funktionen bzw. Request-URLs.

### Eine der Grundlegenden Funktionen ist "getavailableleagues". Diese Funktion hat keine veränderbaren Parameter, sondern gibt (wie der Name schon aussagt) alle in der Datenbank verfügbaren Ligen zurück. Die API-Antwort enthält grundlegende Informationen, wie z.B. den Liga-Namen, die Liga-Saison und die Liga-ID. Letztere ist ein essenzieller Parameter für die Verwendung von vielen weitere Request-URLs. Aufgrund der Menge an verschiedenen Ligen, haben wir uns dafür entschieden uns auf einige wenige zu konzentrieren. Aufgrund der Relevanz und verfügbaren Daten, haben wir uns in dem Projekt für die 1., 2., und 3. Bundesliga entschieden.

### Installation der notwendigen Module

In [1]:
!pip install folium
!pip install requests 
!pip install ipywidgets
!pip install matplotlib

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/be/2e/4dbe5577b7abcc8bc2fdef7e1b0e100ec654ecd47a2bede7ee1516f53f29/folium-0.19.7-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/f8/9d/91cddd38bd00170aad1a4b198c47b4ed716be45c234e09b835af41f4e717/branca-0.8.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.2 MB/s eta 0:00:00


In [2]:
import requests
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import csv
import matplotlib.pyplot as plt
from scipy import stats
from IPython.core.display import HTML

# Forschungsfrage 1 

###  Ziel ist es, eine interaktive Darstellung der Bundesliga-Daten zu erstellen, die es ermöglicht, verschiedene Statistiken (Punkte, Tore, Gegentore, Gewonnene, Verlorene und Unentschiedene) der Teams anzuzeigen. Gleichzeitig sollen statistische Tests durchgeführt werden, um festzustellen, welche Teams signifikant von den Durchschnittswerten abweichen.

In [3]:


# Anlegen einer Datenstruktur, um die verfügbaren Saisons für die jeweilige Bundesliga zu speichern.

bundesliga_data = {
    "bl1": {"name": "1. Bundesliga", "seasons": []},
    "bl2": {"name": "2. Bundesliga", "seasons": []},
    "bl3": {"name": "3. Bundesliga", "seasons": []},
}

# Die einzelnen Funktionen bzw. Forschungsfragen bauen aufeinander auf, deshalb werden in globalen Variablen Informationen wie ausgewählte Liga (current_league), ausgewählte Saison (current_season) ausgewählte Tabelle (current_table) gespeichert.

current_league = None
current_season = None
current_table = None

# Abrufen der verfügbaren Saisons für alle Ligen, anschließend weredn die Ergebnisse gefiltert um nur die relevanten Daten zu erhalten (bei uns 1., 2., und 3. Bundesliga. Die verfügbaren Saisons werden in der Datenstruktur "bundesliga_data" gespeichert, wobei die Saisons in absteigender Reihenfolge sortiert werden. Falls ein Fehler auftritt, wird eine Fehlermeldung ausgegeben.

def fetch_seasons_for_all_ligas():
    url = "https://api.openligadb.de/getavailableleagues"
    try:
        response = requests.get(url)
        response.raise_for_status()
        leagues = response.json()
        for league in leagues:
            shortcut = league['leagueShortcut']
            if shortcut in bundesliga_data:
                # Füge nur Saisons ab 2008 hinzu
                season = league['leagueSeason']
                if int(season) >= 2008:
                    bundesliga_data[shortcut]["seasons"].append(season)

        for shortcut in bundesliga_data:
            bundesliga_data[shortcut]["seasons"].sort(reverse=True)
    except requests.RequestException as e:
        print(f"Fehler beim Abrufen der Daten: {e}")

In [6]:
# Einsetzten der Parameter Liga-Abkürzung (z.B. bl1 für 1. Bundesliga und der Saison (z.B, 2019 für Saison 2019/2020) in die nächste API-URL, um die zugehörige Tabelle abzurufen. Falls ein Fehler auftritt, wird eine Fehlermeldung ausgegeben.

def fetch_table(league_shortcut, season):
    url = f"https://api.openligadb.de/getbltable/{league_shortcut}/{season}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Fehler beim Abrufen der Tabelle: {e}")
        return []

# Visualisierung der abgerufenen Tabelle, Darstellung als Grafik mit dem Modul "Matplotlib". Die Funktion nutzt den Parameter "selected_stat" um auszuwählen, ob die Punkte, Tore, Gegentore, Gewonnen, Verloren oder Unentschieden anzuzeigen. Außerdem wird der Durchschnitt von diesen Parametern angezeigt.

def visualize_table_interactive(df, league_name, season, selected_stat):
    stat_mapping = {
        "Punkte": "points",
        "Tore": "goals",
        "Gegentore": "opponentGoals",
        "Gewonnen": "won",
        "Verloren": "lost",
        "Unentschieden": "draw",
    }

    column_name = stat_mapping[selected_stat]
    avg_value = df[column_name].mean()
    threshold = df[column_name].std()  # Standardabweichung für den Vergleich berechnen

    df_sorted = df.sort_values(by=column_name, ascending=True)

    plt.figure(figsize=(12, 8))
    plt.barh(df_sorted["teamName"], df_sorted[column_name], color="skyblue")
    plt.axvline(avg_value, color="red", linestyle="--", label=f"Durchschnitt: {avg_value:.2f}")
    plt.xlabel(selected_stat, fontsize=12)
    plt.ylabel("Mannschaften", fontsize=12)
    plt.title(f"{selected_stat} - {league_name}, Saison {season}", fontsize=14)
    plt.legend()
    plt.grid(axis="x", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Teams, die signifikant mehr oder weniger Punkte/Tore haben
    significant_teams = []
    for index, row in df.iterrows():
        team_value = row[column_name]
        if team_value > avg_value + threshold:  # Signifikant mehr
            significant_teams.append(f"{row['teamName']} hat signifikant mehr {selected_stat.lower()}.")
        elif team_value < avg_value - threshold:  # Signifikant weniger
            significant_teams.append(f"{row['teamName']} hat signifikant weniger {selected_stat.lower()}.")

    # Ergebnisse ausgeben
    if significant_teams:
        print("Signifikante Abweichungen:")
        for team_result in significant_teams:
            print(team_result)
    else:
        print("Keine signifikanten Abweichungen festgestellt.")
# GUI-Funktionen: Erstellung der Widgets (Dropdownmenü und Knopf zum Bestätigen). Um die Grafiken zu ändern wird die Funktion "update_seasons" genutzt. Die Funktion "on_fetch_button_click" ruft dann die Tabelle für die ausgewählte Liga und Saison ab und speichert sie in der Variable "current_table".

def show_gui():
    fetch_seasons_for_all_ligas()

    league_dropdown = widgets.Dropdown(
        options=[(data["name"], shortcut) for shortcut, data in bundesliga_data.items()],
        description="Liga:"
    )

    season_dropdown = widgets.Dropdown(
        options=[],
        description="Saison:"
    )

    fetch_button = widgets.Button(description="Anzeigen")

    stat_dropdown = widgets.Dropdown(
        options=["Punkte", "Tore", "Gegentore", "Gewonnen", "Verloren", "Unentschieden"],
        description="Statistik:"
    )

    chart_output = widgets.Output()

    def update_seasons(change):
        selected_league = change["new"]
        if selected_league in bundesliga_data:
            season_dropdown.options = bundesliga_data[selected_league]["seasons"]

    def on_fetch_button_click(b):
        global current_league, current_season, current_table
        with chart_output:
            clear_output()
            selected_league = league_dropdown.value
            selected_season = season_dropdown.value
            if selected_league and selected_season:
                table = fetch_table(selected_league, selected_season)
                if table:
                    league_name = bundesliga_data[selected_league]["name"]
                    current_league = selected_league
                    current_season = selected_season
                    current_table = pd.DataFrame(table)
                    visualize_table_interactive(current_table, league_name, selected_season, stat_dropdown.value)
                else:
                    print("Keine Daten für die Tabelle verfügbar.")
            else:
                print("Bitte Liga und Saison auswählen.")

    league_dropdown.observe(update_seasons, names="value")
    fetch_button.on_click(on_fetch_button_click)

    display(widgets.VBox([
        league_dropdown,
        season_dropdown,
        stat_dropdown,
        fetch_button,
        chart_output
    ]))

# GUI anzeigen
show_gui()

# Forschungsfrage 2 

### Auf Basis der in Forschungsfrage 1 ausgewählten Liga und Saison, werden die Mannschaften angezeigt die darin gespielt haben. Wählt man eine Mannschaft aus, so kann man für jeden der 34 Spieltage das Ergebnis sehen. Außerdem kann man sehen ob es ein Heim- oder Auswärtsspiel war und es wird durch eine kleine visuelle Darstellung gezeigt, ob die Mannschaft gewonnen, verloren oder unentschieden gespielt hat.

In [7]:
# Die Matchdaten der ausgewählten Liga und Saison werden durch die Funktion "get_match_data" abgerufen

def get_match_data(league, season):

    url = f"https://api.openligadb.de/getmatchdata/{league}/{season}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Fehler: Nutze die vorherige Funktion, um eine Liga und Saison auszuwählen!")
        return []

# In der Funktion "get_teams_from_matches" werden die Mannschaften aus der API-Antwort extrahiert und mit der zugehörigen TeamID in einem Dictionary gespeichert.

def get_teams_from_matches(match_data):

    teams = {}
    for match in match_data:
        for team in ["team1", "team2"]:
            team_id = match[team]["teamId"]
            team_name = match[team]["teamName"]
            teams[team_id] = team_name
    return teams

# Die Funktion "filter_matches_by_team_and_round" filtert die Spiele für ein Team und einen Spieltag aus den Match-Daten. Die gefilterten Spiele werden in einer Liste gespeichert.

def filter_matches_by_team_and_round(match_data, team_id, round_number):

    filtered_matches = []
    for match in match_data:
        if match["group"]["groupOrderID"] == round_number and (
            match["team1"]["teamId"] == team_id or match["team2"]["teamId"] == team_id
        ):
            filtered_matches.append(match)
    return filtered_matches

# Die Funktion "update_match_display" aktualisiert die Anzeige der Spiele basierend auf dem Dropdown-Menü für die Mannschaften und dem Slider für den ausgewählten Spieltag. Das Team wird mit dem zugehörigen Logo ausgewählt und darunter wird das Ergebnis des Spiels. Außerdem soll mit einem Smiley angezeigt werden, ob das Team gewonnen, verloren oder unentschieden gespielt hat.

def update_match_display(change):

    with output:
        clear_output()
        selected_team = team_dropdown.value
        selected_round = round_slider.value
        filtered_matches = filter_matches_by_team_and_round(match_data, selected_team, selected_round)
        if filtered_matches:
            for match in filtered_matches:
                team1 = match["team1"]
                team2 = match["team2"]
                end_result = next(
                    (result for result in match["matchResults"] if result["resultName"] == "Endergebnis"), None
                )
                if end_result:
                    points_team1 = end_result["pointsTeam1"]
                    points_team2 = end_result["pointsTeam2"]
                    if points_team1 > points_team2:
                        winner = team1
                        loser = team2
                        winner_emoji = "<span style='font-size: 50px;'>😄</span>"
                        loser_emoji = "<span style='font-size: 50px;'>😞</span>"
                    elif points_team2 > points_team1:
                        winner = team2
                        loser = team1
                        winner_emoji = "<span style='font-size: 50px;'>😄</span>"
                        loser_emoji = "<span style='font-size: 50px;'>😞</span>"
                    else:
                        winner = loser = None
                        winner_emoji = loser_emoji = "<span style='font-size: 50px;'>😐</span>"

                    display(HTML(f"""
                    <div style="text-align: center; margin-bottom: 20px;">
                        <div style="display: inline-block; text-align: center; margin-right: 20px;">
                            <img src="{team1['teamIconUrl']}" alt="{team1['teamName']}" style="height: 100px;">
                            <p>{team1['teamName']}</p>
                            { f'<p>{winner_emoji}</p>' if winner == team1 else f'<p>{loser_emoji}</p>' }
                        </div>
                        <div style="display: inline-block; text-align: center; font-size: 20px; font-weight: bold;">
                            VS.
                        </div>
                        <div style="display: inline-block; text-align: center; margin-left: 20px;">
                            <img src="{team2['teamIconUrl']}" alt="{team2['teamName']}" style="height: 100px;">
                            <p>{team2['teamName']}</p>
                            { f'<p>{winner_emoji}</p>' if winner == team2 else f'<p>{loser_emoji}</p>' }
                        </div>
                    </div>
                    <p style="text-align: center; font-size: 24px; font-weight: bold;">
                        Ergebnis: {points_team1} - {points_team2}
                    </p>
                    """))
        else:
            print(f"Keine Spiele für Spieltag {selected_round} gefunden.")

# Die Funktion "show_match_data_gui" ruft die vorherigen Funktionen ab und erstellt das Dropdown-Menü für die Teams sowie den Slider für die Spieltage. Beide sind mit der vorherige Funktion "update_match_display" verknüft damit Änderungen automatisch angezeigt werden.

def show_match_data_gui(league, season):

    global match_data
    match_data = get_match_data(league, season)
    if not match_data:
        print("Keine Match-Daten verfügbar.")
        return

    teams = get_teams_from_matches(match_data)

    global team_dropdown, round_slider, output
    team_dropdown = widgets.Dropdown(
        options=[(name, team_id) for team_id, name in teams.items()],
        description="Team:",
        layout=widgets.Layout(width='300px', align_self='center')
    )
    round_slider = widgets.IntSlider(
        value=1,
        min=1,
        max=34,
        step=1,
        description="Spieltag:",
        continuous_update=False,
        layout=widgets.Layout(width='500px', align_self='center')
    )
    output = widgets.Output()

    team_dropdown.observe(update_match_display, names="value")
    round_slider.observe(update_match_display, names="value")

    display(widgets.VBox([team_dropdown, round_slider, output], layout=widgets.Layout(align_items='center')))

show_match_data_gui(current_league, current_season)

# Forschungsfrage 3 


### Diese Anwendung bietet eine anschauliche Darstellung der geografischen Verteilung der Mannschaften sowie deren sportlicher Statistiken

### In diesem Teil des Projekts möchten wir die Standorte der Mannschaften auf einer Karte visualisieren. Außerdem soll es möglich sein, zwei Teams anhand der Anzahl ihrer gewonnenen Spiele zu vergleichen. Am Ende wird das Ergebnis präsentiert, um zu zeigen, welches Team in diesem Jahr die bessere Leistung erzielt hat.

#### Der Code unten erstellt ein Dictionary mit dem Namen team_locations. In diesem Dictionary sind die Fußballmannschaften aufgeführt, die von 2001 bis 2024 in der ersten, zweiten oder dritten Bundesliga gespielt haben. Zudem sind die Längengrade (Longitude) und Breitengrade (Latitude) der Städte angegeben, die die jeweiligen Mannschaften repräsentieren.

In [8]:
# Stadt und Koordinaten für die Teams
team_locations = {
    "FC St. Pauli": {"city": "Hamburg", "lat": 53.55, "lon": 9.96},
    "MSV Duisburg": {"city": "Duisburg", "lat": 51.43, "lon": 6.76},
    "Wacker Burghausen": {"city": "Burghausen", "lat": 48.16, "lon": 12.87},
    "SG Sonnenhof Großaspach": {"city": "Großaspach", "lat": 48.89, "lon": 9.39},
    "SSV Ulm 1846": {"city": "Ulm", "lat": 48.40, "lon": 9.98},
    "Jahn Regensburg": {"city": "Regensburg", "lat": 49.01, "lon": 12.10},
    "SpVgg Oberfranken Bayreuth": {"city": "Bayreuth", "lat": 49.94, "lon": 11.57},
    "Hannover II": {"city": "Hannover", "lat": 52.37, "lon": 9.73},
    "1. FC Saarbrücken": {"city": "Saarbrücken", "lat": 49.25, "lon": 7.00},
    "Bayer Leverkusen": {"city": "Leverkusen", "lat": 51.00, "lon": 6.99},
    "Fortuna Düsseldorf": {"city": "Düsseldorf", "lat": 51.22, "lon": 6.78},
    "VfB Lübeck": {"city": "Lübeck", "lat": 53.87, "lon": 10.69},
    "SV Darmstadt 98": {"city": "Darmstadt", "lat": 49.87, "lon": 8.65},
    "FC Bayern München": {"city": "München", "lat": 48.14, "lon": 11.58},
    "KFC Uerdingen 05": {"city": "Krefeld", "lat": 51.34, "lon": 6.56},
    "Arminia Bielefeld": {"city": "Bielefeld", "lat": 50.48, "lon": 8.53},
    "RB Leipzig": {"city": "Leipzig", "lat": 51.34, "lon": 12.37},
    "SV Wehen Wiesbaden": {"city": "Wiesbaden", "lat": 50.08, "lon": 8.25},
    "SC Freiburg II": {"city": "Freiburg im Breisgau", "lat": 47.99, "lon": 7.85},
    "Rot-Weiss Oberhausen": {"city": "Oberhausen", "lat": 51.48, "lon": 6.85},
    "Hertha BSC": {"city": "Berlin", "lat": 52.52, "lon": 13.41},
    "SV Waldhof Mannheim": {"city": "Mannheim", "lat": 49.48, "lon": 8.46},
    "SC Freiburg": {"city": "Freiburg im Breisgau", "lat": 47.99, "lon": 7.85},
    "Holstein Kiel": {"city": "Kiel", "lat": 54.32, "lon": 10.13},
    "Sportfreunde Lotte": {"city": "Lotte", "lat": 52.45, "lon": 7.88},
    "1. FC Heidenheim 1846": {"city": "Heidenheim an der Brenz", "lat": 48.66, "lon": 10.15},
    "TSG 1899 Hoffenheim": {"city": "Hoffenheim", "lat": 49.14, "lon": 8.83},
    "Kickers Emden": {"city": "Emden", "lat": 53.37, "lon": 7.21},
    "Chemnitzer FC": {"city": "Chemnitz", "lat": 50.83, "lon": 12.92},
    "Werder Bremen": {"city": "Bremen", "lat": 53.08, "lon": 8.80},
    "FC Schalke 04": {"city": "Gelsenkirchen", "lat": 51.50, "lon": 7.20},
     "1. FC Kaiserslautern": {"city": "Kaiserslautern", "lat": 49.44, "lon": 7.75},
    "Kickers Offenbach": {"city": "Offenbach am Main", "lat": 50.09, "lon": 8.77},
    "Würzburger Kickers": {"city": "Würzburg", "lat": 49.79, "lon": 9.93},
    "Fortuna Köln": {"city": "Köln", "lat": 50.94, "lon": 6.96},
    "SV Elversberg 07": {"city": "Elversberg", "lat": 49.31, "lon": 7.15},
    "FC Erzgebirge Aue": {"city": "Aue", "lat": 50.59, "lon": 12.67},
    "FC Kickers Würzburg": {"city": "Würzburg", "lat": 49.79, "lon": 9.93},
    "Borussia Mönchengladbach": {"city": "Mönchengladbach", "lat": 51.18, "lon": 6.43},
    "Karlsruher SC": {"city": "Karlsruhe", "lat": 49.01, "lon": 8.39},
    "1. FSV Mainz 05 II": {"city": "Mainz", "lat": 50.00, "lon": 8.27},
    "VfL Bochum": {"city": "Bochum", "lat": 51.48, "lon": 7.21},
    "FC Viktoria Köln": {"city": "Köln", "lat": 50.94, "lon": 6.96},
    "1. FSV Mainz 05": {"city": "Mainz", "lat": 50.00, "lon": 8.27},
    "Hamburger SV": {"city": "Hamburg", "lat": 53.55, "lon": 9.96},
    "FC Bayern München II": {"city": "München", "lat": 48.14, "lon": 11.58},
    "FC Ingolstadt 04": {"city": "Ingolstadt", "lat": 48.74, "lon": 11.43},
    "1. FC Nürnberg": {"city": "Nürnberg", "lat": 49.45, "lon": 11.06},
    "VfL Osnabrück": {"city": "Osnabrück", "lat": 52.28, "lon": 8.05},
    "Borussia Dortmund": {"city": "Dortmund", "lat": 51.51, "lon": 7.46},
    "SV Sandhausen": {"city": "Sandhausen", "lat": 49.35, "lon": 8.63},
      "Stuttgarter Kickers": {"city": "Stuttgart", "lat": 48.77, "lon": 9.18},
    "SV Babelsberg 03": {"city": "Potsdam", "lat": 52.39, "lon": 12.93},
    "Preußen Münster": {"city": "Münster", "lat": 51.96, "lon": 7.63},
    "SG Dynamo Dresden": {"city": "Dresden", "lat": 51.05, "lon": 13.73},
    "Werder Bremen II": {"city": "Bremen", "lat": 53.08, "lon": 8.80},
    "Erzgebirge Aue": {"city": "Aue", "lat": 50.59, "lon": 12.67},
    "Hannover 96": {"city": "Hannover", "lat": 52.37, "lon": 9.73},
    "Borussia Dortmund II": {"city": "Dortmund", "lat": 51.51, "lon": 7.46},
    "SC Paderborn 07": {"city": "Paderborn", "lat": 51.72, "lon": 8.75},
    "1. FC Köln": {"city": "Köln", "lat": 50.94, "lon": 6.96},
    "1. FC Magdeburg": {"city": "Magdeburg", "lat": 52.13, "lon": 11.63},
    "Eintracht Frankfurt": {"city": "Frankfurt am Main", "lat": 50.11, "lon": 8.68},
    "SC Verl": {"city": "Verl", "lat": 51.91, "lon": 8.63},
    "Türkgücü München": {"city": "München", "lat": 48.14, "lon": 11.58},
    "FC Rot-Weiß Erfurt": {"city": "Erfurt", "lat": 50.98, "lon": 11.03},
    "Rot-Weiß Erfurt": {"city": "Erfurt", "lat": 50.98, "lon": 11.03},  # Duplikat
    "FC Viktoria 1889 Berlin": {"city": "Berlin", "lat": 52.52, "lon": 13.41},
    "VfB Oldenburg": {"city": "Oldenburg", "lat": 53.14, "lon": 8.21},
    "VfB Stuttgart II": {"city": "Stuttgart", "lat": 48.77, "lon": 9.18},
    "TSG 1899 Hoffenheim": {"city": "Hoffenheim", "lat": 49.14, "lon": 8.83},
    "Rot-Weiss Ahlen": {"city": "Ahlen", "lat": 51.75, "lon": 8.07},
    "Alemannia Aachen": {"city": "Aachen", "lat": 50.77, "lon": 6.09},
    "SC Fortuna Köln": {"city": "Köln", "lat": 50.94, "lon": 6.96},
    "FC Augsburg": {"city": "Augsburg", "lat": 48.40, "lon": 10.98},
    "SpVgg Bayreuth": {"city": "Bayreuth", "lat": 49.94, "lon": 11.57},
    "Rot-Weiss Essen": {"city": "Essen", "lat": 51.45, "lon": 7.02},
    "SV Meppen": {"city": "Meppen", "lat": 52.69, "lon": 7.29},
    "FC Energie Cottbus": {"city": "Cottbus", "lat": 51.76, "lon": 14.33},
    "VfL Wolfsburg": {"city": "Wolfsburg", "lat": 52.42, "lon": 10.79},
    "TuS Koblenz": {"city": "Koblenz", "lat": 50.36, "lon": 7.58},
     "FC Hansa Rostock": {"city": "Rostock", "lat": 54.09, "lon": 12.08},
    "FSV Zwickau": {"city": "Zwickau", "lat": 50.70, "lon": 12.50},
    "Wuppertaler SV": {"city": "Wuppertal", "lat": 51.27, "lon": 7.19},
    "Eintracht Braunschweig": {"city": "Braunschweig", "lat": 52.26, "lon": 10.52},
    "TSV Havelse": {"city": "Garbsen", "lat": 52.44, "lon": 9.58},
    "VfB Stuttgart": {"city": "Stuttgart", "lat": 48.77, "lon": 9.18},
    "1. FC Union Berlin": {"city": "Berlin", "lat": 52.52, "lon": 13.49},
    "VfR Aalen": {"city": "Aalen", "lat": 48.84, "lon": 10.09},
    "SpVgg Greuther Fürth": {"city": "Fürth", "lat": 49.47, "lon": 10.99},
    "SpVgg Unterhaching": {"city": "Unterhaching", "lat": 48.02, "lon": 11.59},
    "TSV 1860 München": {"city": "München", "lat": 48.14, "lon": 11.58},
    "FC Carl Zeiss Jena": {"city": "Jena", "lat": 50.93, "lon": 11.58},
    "FSV Frankfurt": {"city": "Frankfurt am Main", "lat": 50.11, "lon": 8.68},
    "Hallescher FC": {"city": "Halle (Saale)", "lat": 51.47, "lon": 11.97},
    "SSV Ulm 1846": {"city": "Ulm", "lat": 48.40, "lon": 9.98},
}

#### Diese Funktion ist erforderlich für die spätere Darstellung der Teams und die URLs der Logos, die für die visuelle Präsentation im Graphen benötigt werden. In diesem Codeabschnitt rufen wir eine API ab und erstellen eine CSV-Datei, die die Teamnamen, die kurzen Namen der Mannschaften und die Links zu deren Logos enthält.

In [9]:

# Datei zur Speicherung der Teams
output_file = "bundesliga_teams_with_images.csv"

# Liga-Kürzel
bundesliga_ids = ["bl1", "bl2", "bl3"]
jahre = range(2008, 2025)  # Jahre von 2008 bis 2024

# Set für Duplikate
unique_teams = set()

# Loop durch die Ligen und Jahre
for liga in bundesliga_ids:
    for jahr in jahre:
        url = f"https://api.openligadb.de/getavailableteams/{liga}/{jahr}"
        response = requests.get(url)
        
        if response.status_code == 200:
            teams = response.json()
            # Extrahiere teamName, shortName und teamIconUrl
            for team in teams:
                team_name = team["teamName"]
                short_name = team["shortName"]
                team_icon_url = team["teamIconUrl"]
                unique_teams.add((team_name, short_name, team_icon_url))
        else:
            print(f"Fehler beim Abrufen der Teams für {liga} im Jahr {jahr}: {response.status_code}")

# Speichern der einzigartigen Teams in einer CSV-Datei
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Teamname", "Shortname", "Bild-URL"])  # Kopfzeile einfügen
    
    for team in unique_teams:
        writer.writerow(team)

print(f"Die Teams wurden erfolgreich gespeichert in {output_file}.")

Die Teams wurden erfolgreich gespeichert in bundesliga_teams_with_images.csv.


#### Wir laden den Inhalt der zuvor gespeicherten CSV-Datei und übertragen die Spalte, die die Links zu den Logos der Teams enthält, in das Dictionary namens team_locations. Am Ende erhalten wir ein aktualisiertes Dictionary, das die Teamnamen, die Koordinaten und die Städte der Mannschaften umfasst.

In [10]:


# CSV-Datei laden
csv_file_path = 'bundesliga_teams_with_images.csv'  # Pfad zur CSV-Datei
data = pd.read_csv(csv_file_path)

# Erstelle ein Dictionary für die Teamnamen und ihre Bild-URLs
team_image_urls = {row['Teamname']: row['Bild-URL'] for index, row in data.iterrows()}


# Aktualisiere die team_locations mit den Bild-URLs
for team, location in team_locations.items():
    if team in team_image_urls:
        location['image_url'] = team_image_urls[team]  # Füge die Bild-URL hinzu
    else:
        location['image_url'] = None  # Falls kein Bild-URL vorhanden ist


#### In diesem Abschnitt wird eine interaktive Anwendung entwickelt, die es Benutzern ermöglicht, Daten über Fußballmannschaften aus den drei deutschen Ligen abzurufen. Zu Beginn werden die verfügbaren Saisons ab 2008 (da für vorherige Jahre keine Spielerdaten vorliegen) von einer API abgerufen und in einer strukturierten Datenbank gespeichert. Benutzer können Ligen und Jahre über Dropdown-Menüs auswählen, wodurch eine Karte mit den Teams und deren Standorten angezeigt wird. Marker auf der Karte repräsentieren die Teams und zeigen beim Klicken Informationen wie die Anzahl der gewonnenen, verlorenen und unentschiedenen Spiele an. Zusätzlich haben die Benutzer die Möglichkeit, unter der Karte zwei Teams auszuwählen, um deren Leistungen zu vergleichen.

### Die Funktion erstellt eine auf Deutschland zentrierte Karte mit der Folium-Bibliothek und ruft Daten zu den Fußballteams aus einer API ab. Nach dem erfolgreichen Abruf werden die Teamnamen in Dropdown-Menüs für den Vergleich eingepflegt. Für jedes Team wird ein Marker auf der Karte platziert, der den Teamnamen und die Ergebnisse (Gewonnen, Verloren, Unentschieden) zeigt. Diese Informationen sind in einem Popup sichtbar, wenn der Benutzer den Marker anklickt. Schließlich wird die fertige Karte angezeigt.


In [11]:
# Initialer Abruf der Saisons
fetch_seasons_for_all_ligas()

# Erstelle Dropdown für Ligen
liga_dropdown = widgets.Dropdown(
    options=[(bundesliga_data[key]["name"], key) for key in bundesliga_data],
    description='Liga:'
)

# Erstelle Dropdown für Jahre
jahr_dropdown = widgets.Dropdown(
    options=bundesliga_data[liga_dropdown.value]["seasons"],
    description='Jahr:'
)

# Output-Widget zur Anzeige der Karte
output_map = widgets.Output()

# Dropdown-Menüs für Team-Vergleich
team1_dropdown = widgets.Dropdown(description='Team 1:')
team2_dropdown = widgets.Dropdown(description='Team 2:')

# Ausgabe-Widget für Ergebnisse
output_comparison = widgets.Output()

# Gruppe von Teams
all_teams = []

# Funktion zum Abrufen und Anzeigen der Karte
def fetch_data_and_display_map(liga, jahr):
    global all_teams
    with output_map:
        clear_output()
        
        # Erstellen Sie eine Folium-Karte, die auf Deutschland zentriert ist
        m = folium.Map(location=[51.1657, 10.4515], zoom_start=6)  # Deutschland-Zentrum

        # API-Anfrage für die Teamdaten
        url = f'https://api.openligadb.de/getbltable/{liga}/{jahr}'
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            all_teams = [team['teamName'] for team in data]  # Alle Teamnamen sammeln
            team1_dropdown.options = all_teams  # Dropdown-Optionen setzen
            team2_dropdown.options = all_teams  # Dropdown-Optionen setzen
            
            for team in data:
                team_name = team['teamName']
                won = team['won']
                lost = team['lost']
                draw = team['draw']
                
                # Überprüfen, ob das Team in team_locations vorhanden ist
                if team_name in team_locations:
                    location = team_locations[team_name]
                    
                    # Benutzerdefiniertes Icon verwenden
                    icon = folium.CustomIcon(icon_image=location['image_url'], icon_size=(60, 60))

                    # Marker mit benutzerdefiniertem Icon und Team-Informationen hinzufügen
                    folium.Marker(
                        location=[location['lat'], location['lon']],
                        icon=icon,
                        popup=folium.Popup(
                            f"<b>{team_name}</b><br>"
                            f"Gewonnen: {won}<br>"
                            f"Verloren: {lost}<br>"
                            f"Unentschieden: {draw}",
                            max_width=200
                        ),
                        tooltip=team_name
                    ).add_to(m)

            # Karte anzeigen
            display(m)
        else:
            print("Fehler beim Abrufen der Daten:", response.status_code)
            
            

#### Die Funktion on_change_liga aktualisiert die verfügbaren Jahre im Dropdown-Menü, wenn der Benutzer eine neue Liga auswählt, und setzt das erste Jahr als Standardwert. Die Funktion on_change_jahr ruft die Kartenanzeige auf, wenn sich das ausgewählte Jahr ändert. Beide Funktionen sind so eingerichtet, dass sie auf Änderungen in den Dropdown-Menüs reagieren.

In [12]:
def on_change_liga(change):
    new_liga = change['new']
    jahre = bundesliga_data[new_liga]["seasons"]
    jahr_dropdown.options = jahre
    jahr_dropdown.value = jahre[0] if jahre else None

def on_change_jahr(change):
    fetch_data_and_display_map(liga_dropdown.value, jahr_dropdown.value)

liga_dropdown.observe(on_change_liga, names='value')
jahr_dropdown.observe(on_change_jahr, names='value')


#### Der Code fügt einen "Vergleichen"-Button hinzu, der es Benutzern ermöglicht, zwei Fußballteams zu vergleichen. Bei einem Klick sendet die Funktion compare_teams eine Anfrage an die API, um die Anzahl der gewonnenen Spiele beider Teams abzurufen und das Ergebnis anzuzeigen. Bei Fehlern oder nicht gefundenen Teams werden entsprechende Meldungen ausgegeben.

In [13]:
# Vergleichsbutton hinzufügen
compare_button = widgets.Button(description="Vergleichen")


# Vergleichsfunktion
def compare_teams(b):
    team1 = team1_dropdown.value
    team2 = team2_dropdown.value
    output_comparison.clear_output()  # Vorherige Ausgabe löschen
    with output_comparison:
        if team1 and team2:
            url = f'https://api.openligadb.de/getbltable/{liga_dropdown.value}/{jahr_dropdown.value}'
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                team1_data = next((team for team in data if team['teamName'] == team1), None)
                team2_data = next((team for team in data if team['teamName'] == team2), None)
                
                if team1_data and team2_data:
                    team1_won = team1_data['won']
                    team2_won = team2_data['won']
                    
                    if team1_won > team2_won:
                        display(f"{team1} hat mehr Spiele gewonnen als {team2}.")
                    elif team1_won < team2_won:
                        display(f"{team2} hat mehr Spiele gewonnen als {team1}.")
                    else:
                        display(f"{team1} und {team2} haben gleich viele Spiele gewonnen.")
                else:
                    display("Fehler: Ein oder beide Teams wurden nicht gefunden!")
            else:
                display("Fehler beim Abrufen der Daten:", response.status_code)

compare_button.on_click(compare_teams)

# Anzeige der Widgets
display(liga_dropdown, jahr_dropdown, output_map, team1_dropdown, team2_dropdown, compare_button, output_comparison)

# Erstmalige Daten abrufen und Karte anzeigen
fetch_data_and_display_map(liga_dropdown.value, jahr_dropdown.value)

Dropdown(description='Liga:', options=(('1. Bundesliga', 'bl1'), ('2. Bundesliga', 'bl2'), ('3. Bundesliga', '…

Dropdown(description='Jahr:', options=('2025', '2025', '2025', '2024', '2024', '2024', '2023', '2023', '2023',…

Output()

Dropdown(description='Team 1:', options=(), value=None)

Dropdown(description='Team 2:', options=(), value=None)

Button(description='Vergleichen', style=ButtonStyle())

Output()